In [1]:
import serial
import csv

arduino_port = "COM5" #serial port of Arduino
baud = 115200 #arduino uno runs at 9600 baud
fileName="analog-data.csv" #name of the CSV file generated
ser = serial.Serial(arduino_port, baud)



In [ ]:
import csv
import serial
from time import sleep

arduino_port = "COM5"  # Replace with your actual port
fileName = "sensor_data.csv"

print("Connected to Arduino port: " + arduino_port)
file = open(fileName, "a")
print("Created file")

samples = 200
line = 0
sensor_data = []

# Column names for 5 sensors + label
column_names = ["Flex1", "Flex2", "Flex3", "Flex4", "Flex5",
                "accel_x","accel_y","accel_z","gyro_x","gyro_y","gyro_z",
                "Label"]
sensor_data.append(column_names)

# Start serial connection
ser = serial.Serial(arduino_port, 115200)

# Countdown before starting
print("Starting in:")
for i in range(5, 0, -1):
    print(i)
    sleep(1)
print("Start collecting data...")

boo = True

while line < samples:
    getData = ser.readline().decode().strip()
    readings = getData.split(",")

    # Ensure correct number of readings
    if len(readings) != 11:
        continue  # skip invalid lines

    # Add label and notify user
    if line < samples / 2:
        label = "normal"
        if line == 0:
            print("Label now: normal")
    else:
        label = "กำหมัด"
        if line == samples // 2:
            print("Label now: กำหมัด")
            print("Please กำหมัด (make a fist) now!")
            sleep(2)  # Wait for user to act

    readings.append(label)
    sensor_data.append(readings)
    print(readings)

    line += 1

# Save to CSV
with open(fileName, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(sensor_data)

print("Data collection complete!")
file.close()


In [2]:
import pandas as pd

df = pd.read_csv('sensor_data.csv')
df

,timestamp_ms,motion_detected,roll_deg,pitch_deg,yaw_deg,filtered_accel_x_(m/s²),filtered_accel_y_(m/s²),filtered_accel_z_(m/s²),filtered_gyro_x_(rad/s),filtered_gyro_y_(rad/s),filtered_gyro_z_(rad/s),accel_magnitude,gyro_magnitude,adc0,adc1,adc2,adc3,adc4,Label
0,24878,0,1.645825,-0.339998,0.000000,0.005832,0.028226,0.982339,-0.324456,0.321475,-0.157242,9.827621,4.830561,833,840,862,898,874,normal
1,24905,0,1.712330,-0.464993,0.005518,0.015151,0.055784,1.865966,-0.644592,0.556115,0.204365,9.823804,5.613591,833,840,863,898,874,normal
2,24932,0,1.776976,-0.744087,0.006702,0.034551,0.082502,2.659075,-0.804589,0.823541,0.043874,9.804610,4.175514,833,840,858,894,873,normal
3,24959,0,1.883953,-0.829099,0.014618,0.048899,0.111097,3.377183,-0.798586,0.962662,0.293182,9.848659,3.449003,832,840,862,898,874,normal
4,24986,0,1.907530,-0.825511,0.023783,0.057982,0.133960,4.021804,-0.591621,0.951933,0.339434,9.830259,1.708319,833,840,863,898,874,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,32676,0,0.817209,14.445817,-10.949023,-2.700550,0.154397,10.482001,1.410927,-0.518520,0.514201,10.928462,3.705091,737,641,703,835,767,กำหมัด
196,32703,0,0.784510,14.424902,-10.926008,-2.697612,0.148269,10.486532,1.148503,-0.154568,0.852413,10.861315,5.137523,737,641,703,835,768,กำหมัด
197,32730,0,0.803768,14.442130,-10.901229,-2.699756,0.151851,10.481751,0.812321,0.221427,0.917742,10.788611,4.490503,737,642,703,835,768,กำหมัด
198,32757,0,0.823623,14.387828,-10.868830,-2.688997,0.155554,10.481040,0.814445,0.411384,1.199976,10.792268,4.379686,737,642,703,835,768,กำหมัด


In [ ]:
import threading
import tkinter as tk
from tkinter import scrolledtext, messagebox, font as tkfont, ttk
import serial
import serial.tools.list_ports
import time

# ─── CONFIGURATION ──────────────────────────────────────────────────────────
BAUDRATE        = 9600         # Must match Serial.begin(...) in your Arduino/ESP sketch
FONT_FAMILY     = 'Consolas'   # e.g. "Consolas", "Courier", "Helvetica"
FONT_SIZE_TEXT  = 11           # Font size for text areas
FONT_SIZE_ENTRY = 12           # Font size for the input field
# ─────────────────────────────────────────────────────────────────────────────


class SerialGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("Serial Monitor (Commands & Sensor Data)")

        # ─── INTERNAL STATE ───────────────────────────────────────────────────────
        self.ser = None
        self.alive = False
        self.reader_thread = None
        self.sensor_paused = False

        # ─── Main window layout ──────────────────────────────────────────────────
        # Row 0: Port selection controls + Connect/Disconnect
        # Row 1: PanedWindow (VERTICAL) with:
        #         • Top pane: Commands / Status
        #         • Bottom pane: Sensor Data + Pause/Resume
        # Row 2: Entry & Send button
        # Row 3: Close button
        self.master.rowconfigure(0, weight=0)
        self.master.rowconfigure(1, weight=1)
        self.master.rowconfigure(2, weight=0)
        self.master.rowconfigure(3, weight=0)

        # Two columns: Column 0 expands; Column 1 holds right-aligned buttons
        self.master.columnconfigure(0, weight=1)
        self.master.columnconfigure(1, weight=0)

        # ─── Create Fonts ───────────────────────────────────────────────────────
        self.text_font  = tkfont.Font(family=FONT_FAMILY, size=FONT_SIZE_TEXT)
        self.entry_font = tkfont.Font(family=FONT_FAMILY, size=FONT_SIZE_ENTRY)

        # ─── Row 0: Port Selection Frame ─────────────────────────────────────────
        port_frame = tk.Frame(master)
        port_frame.grid(row=0, column=0, columnspan=2, sticky="we", padx=10, pady=(10, 0))
        port_frame.columnconfigure(0, weight=0)
        port_frame.columnconfigure(1, weight=1)
        port_frame.columnconfigure(2, weight=0)
        port_frame.columnconfigure(3, weight=0)
        port_frame.columnconfigure(4, weight=0)

        tk.Label(port_frame, text="Port:", font=self.entry_font).grid(row=0, column=0, sticky="w")
        self.port_var = tk.StringVar()
        self.port_combo = ttk.Combobox(
            port_frame,
            textvariable=self.port_var,
            state='readonly',
            font=self.entry_font
        )
        self.port_combo.grid(row=0, column=1, sticky="we", padx=(5, 5))

        self.refresh_button = tk.Button(
            port_frame,
            text="Refresh",
            font=self.entry_font,
            command=self.refresh_ports
        )
        self.refresh_button.grid(row=0, column=2, sticky="e", padx=(5, 5))

        self.connect_button = tk.Button(
            port_frame,
            text="Connect",
            font=self.entry_font,
            command=self.connect_serial
        )
        self.connect_button.grid(row=0, column=3, sticky="e", padx=(5, 5))

        self.disconnect_button = tk.Button(
            port_frame,
            text="Disconnect",
            font=self.entry_font,
            command=self.disconnect_serial,
            state=tk.DISABLED  # initially disabled
        )
        self.disconnect_button.grid(row=0, column=4, sticky="e")

        # Populate initial port list
        self.refresh_ports()

        # ─── Row 1: PanedWindow (VERTICAL) ────────────────────────────────────────
        self.main_pane = tk.PanedWindow(master, orient=tk.VERTICAL)
        self.main_pane.grid(row=1, column=0, columnspan=2, sticky="nsew", padx=10, pady=5)

        # --- Top pane: Commands / Status ---
        top_frame = tk.Frame(self.main_pane)
        top_frame.rowconfigure(0, weight=0)  # Label
        top_frame.rowconfigure(1, weight=1)  # Text area
        top_frame.columnconfigure(0, weight=1)

        lbl_top = tk.Label(
            top_frame,
            text="Commands / Status",
            font=(FONT_FAMILY, FONT_SIZE_TEXT + 2, "bold")
        )
        lbl_top.grid(row=0, column=0, sticky="w", pady=(0, 4))

        self.top_text = scrolledtext.ScrolledText(
            top_frame,
            wrap=tk.WORD,
            font=self.text_font,
            state=tk.DISABLED,
            bg='black',
            fg='lightyellow'
        )
        self.top_text.grid(row=1, column=0, sticky="nsew")

        self.main_pane.add(top_frame, minsize=200)

        # --- Bottom pane: Sensor Data + Pause Button ---
        bottom_frame = tk.Frame(self.main_pane)
        bottom_frame.rowconfigure(0, weight=0)  # Pause button
        bottom_frame.rowconfigure(1, weight=0)  # Label
        bottom_frame.rowconfigure(2, weight=1)  # Text area
        bottom_frame.columnconfigure(0, weight=1)

        self.pause_button = tk.Button(
            bottom_frame,
            text="Pause Sensor",
            font=self.entry_font,
            command=self.toggle_pause_sensor
        )
        self.pause_button.grid(row=0, column=0, sticky="w", pady=(0, 6))

        lbl_bottom = tk.Label(
            bottom_frame,
            text="Sensor Data",
            font=(FONT_FAMILY, FONT_SIZE_TEXT + 2, "bold")
        )
        lbl_bottom.grid(row=1, column=0, sticky="w", pady=(0, 4))

        self.bottom_text = scrolledtext.ScrolledText(
            bottom_frame,
            wrap=tk.WORD,
            font=self.text_font,
            state=tk.DISABLED,
            bg='black',
            fg='cyan'
        )
        self.bottom_text.grid(row=2, column=0, sticky="nsew")

        self.main_pane.add(bottom_frame, minsize=150)

        # ─── Row 2: Entry Field & Send Button ─────────────────────────────────────
        self.entry = tk.Entry(master, font=self.entry_font, state=tk.DISABLED)
        self.entry.grid(
            row=2, column=0,
            padx=(10, 5), pady=(0, 10),
            sticky="we"
        )
        self.entry.bind("<Return>", lambda event: self.send_data())

        self.send_button = tk.Button(
            master,
            text="Send",
            font=self.entry_font,
            command=self.send_data,
            state=tk.DISABLED,
            width=10
        )
        self.send_button.grid(
            row=2, column=1,
            padx=(0, 10), pady=(0, 10),
            sticky="e"
        )

        # ─── Row 3: Close Button ─────────────────────────────────────────────────
        self.close_button = tk.Button(
            master,
            text="Close",
            font=self.entry_font,
            command=self.close
        )
        self.close_button.grid(
            row=3, column=0, columnspan=2,
            pady=(0, 10)
        )

        # Ensure window closes cleanly
        self.master.protocol("WM_DELETE_WINDOW", self.close)

    # ─── Port-Scanning & Connection ─────────────────────────────────────────────

    def refresh_ports(self):
        """
        Re-scan available serial ports and update the combobox.
        If already connected, send 'reset' to the board.
        """
        ports = [p.device for p in serial.tools.list_ports.comports()]
        self.port_combo['values'] = ports
        if ports:
            if self.port_var.get() not in ports:
                self.port_var.set(ports[0])
        else:
            self.port_var.set('')

        # If serial is already open, send "reset"
        if self.ser and self.ser.is_open:
            try:
                self.ser.write(("reset\n").encode())
                self._append_to_top(f"[You] reset\n")
            except serial.SerialException as e:
                messagebox.showerror("Write Error", f"Failed to send reset:\n{e}")

    def connect_serial(self):
        """
        Open the selected port, enable panes & send controls, then send 'reset'.
        """
        selected = self.port_var.get().strip()
        if not selected:
            messagebox.showwarning("No Port Selected", "Please select a serial port first.")
            return

        try:
            self.ser = serial.Serial(selected, BAUDRATE, timeout=1)
            time.sleep(2)  # Give the board a moment to reset
        except serial.SerialException as e:
            messagebox.showerror("Connection Error", f"Could not open {selected!r}:\n{e}")
            return

        # Disable port-selection controls
        self.port_combo.config(state='disabled')
        self.refresh_button.config(state='disabled')
        self.connect_button.config(state='disabled')
        # Enable Disconnect button
        self.disconnect_button.config(state=tk.NORMAL)

        # Enable entry + send button
        self.entry.config(state=tk.NORMAL)
        self.send_button.config(state=tk.NORMAL)

        # Clear both panes, then show connected notice
        self._clear_panes()
        self._append_to_top(f"[Arduino] *** Connected to {selected} @ {BAUDRATE} baud ***\n")

        # Immediately send "reset" to the board
        try:
            self.ser.write(("reset\n").encode())
            self._append_to_top(f"[You] reset\n")
        except serial.SerialException as e:
            messagebox.showerror("Write Error", f"Failed to send reset:\n{e}")

        # Start background reading thread
        self.alive = True
        self.reader_thread = threading.Thread(target=self.read_from_serial, daemon=True)
        self.reader_thread.start()

    def disconnect_serial(self):
        """
        Disconnect from the serial port without closing the GUI.
        Stops the reader thread, closes the port, and re-enables controls.
        """
        if self.ser and self.ser.is_open:
            self.alive = False
            try:
                self.ser.close()
            except Exception:
                pass

        # Disable send controls
        self.entry.config(state=tk.DISABLED)
        self.send_button.config(state=tk.DISABLED)

        # Re-enable port-selection controls
        self.port_combo.config(state='readonly')
        self.refresh_button.config(state=tk.NORMAL)
        self.connect_button.config(state=tk.NORMAL)
        self.disconnect_button.config(state=tk.DISABLED)

        # Inform user
        self._append_to_top("[Arduino] *** Disconnected ***\n")

    # ─── Background Reader & Routing ────────────────────────────────────────────

    def read_from_serial(self):
        """
        Continuously read lines from serial and route:
        - If line.startswith("[Sensor]")   → bottom pane (unless paused)
        - Else                               → top pane
        """
        while self.alive:
            try:
                if self.ser.in_waiting > 0:
                    raw = self.ser.readline().decode(errors='ignore').rstrip('\r\n')
                    if not raw:
                        continue

                    if raw.startswith("[Sensor]"):
                        # Sensor data
                        if not self.sensor_paused:
                            self._append_to_bottom(raw + "\n")
                    else:
                        # Anything else (including "[Arduino]" or unprefixed) → top pane
                        self._append_to_top(raw + "\n")
                else:
                    time.sleep(0.02)
            except (serial.SerialException, OSError):
                break

    # ─── Sending Data ───────────────────────────────────────────────────────────

    def send_data(self):
        """
        When user presses Enter or clicks "Send":
        - Send "<text>\n" to the board
        - Echo "[You] <text>" in the top pane
        """
        text = self.entry.get().strip()
        if not text or self.ser is None or not self.ser.is_open:
            return

        try:
            self.ser.write((text + "\n").encode())
            self._append_to_top(f"[You] {text}\n")
        except serial.SerialException as e:
            messagebox.showerror("Write Error", f"Failed to write to serial port:\n{e}")
            return

        self.entry.delete(0, tk.END)

    # ─── Pause/Resume Sensor Pane ────────────────────────────────────────────

    def toggle_pause_sensor(self):
        """
        Toggle whether sensor lines are appended to the bottom pane.
        """
        self.sensor_paused = not self.sensor_paused
        if self.sensor_paused:
            self.pause_button.config(text="Resume Sensor")
        else:
            self.pause_button.config(text="Pause Sensor")

    # ─── Closing ─────────────────────────────────────────────────────────────

    def close(self):
        """
        Stop reader thread, close the port, then destroy the window.
        """
        if messagebox.askokcancel("Quit", "Do you really want to quit?"):
            self.alive = False
            try:
                if self.ser and self.ser.is_open:
                    self.ser.close()
            except Exception:
                pass
            self.master.destroy()

    # ─── Helpers to Append/Clear Panes ───────────────────────────────────────────

    def _append_to_top(self, text: str):
        """
        Schedule appending `text` to the top “Commands / Status” pane.
        """
        def inner():
            self.top_text.configure(state=tk.NORMAL)
            self.top_text.insert(tk.END, text)
            self.top_text.see(tk.END)
            self.top_text.configure(state=tk.DISABLED)
        self.master.after(0, inner)

    def _append_to_bottom(self, text: str):
        """
        Schedule appending `text` to the bottom “Sensor Data” pane.
        """
        def inner():
            self.bottom_text.configure(state=tk.NORMAL)
            self.bottom_text.insert(tk.END, text)
            self.bottom_text.see(tk.END)
            self.bottom_text.configure(state=tk.DISABLED)
        self.master.after(0, inner)

    def _clear_panes(self):
        """
        Clear both the top and bottom ScrolledText widgets.
        """
        def inner():
            for pane in (self.top_text, self.bottom_text):
                pane.configure(state=tk.NORMAL)
                pane.delete('1.0', tk.END)
                pane.configure(state=tk.DISABLED)
        self.master.after(0, inner)


if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("800x600")  # Adjust initial size as needed
    gui = SerialGUI(root)
    root.mainloop()


Exception in thread Thread-18:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.7_3.7.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.7_3.7.2544.0_x64__qbz5n2kfra8p0\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Sumat\AppData\Local\Temp\ipykernel_29876\1091606856.py", line 266, in read_from_serial
    self._append_to_bottom(raw + "\n")
  File "C:\Users\Sumat\AppData\Local\Temp\ipykernel_29876\1091606856.py", line 345, in _append_to_bottom
    self.master.after(0, inner)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.7_3.7.2544.0_x64__qbz5n2kfra8p0\lib\tkinter\__init__.py", line 757, in after
    return self.tk.call('after', ms, name)
RuntimeError: main thread is not in main loop

